# Pulling data from the advertising spots endpoint using pybarb

In this demo we will show you how to pull data and then manipluate data from the advertising spots endpoint using the pybarb library. To show this we will demonstrate a typical media agency use case in which an agency has run a TV advertising campaign for their client. As part of their post campaign analysis they would like to understand how well the adverts have performed

1. By TV station
2. By audience 
3. And over time.   

Note the full API documentation can be found [here](https://barb-api.co.uk/api-docs). 

It might also be useful to consult the [Getting Started](https://barb-api.co.uk/api-docs#section/Getting-started) section for information about authentication and basic API usage.


## Querying the API with pybarb

First we connect to the API using the `pybarb` package as described in "Connecting to the Barb API using Python". 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/Users/simon_business/Documents/code_repos/barb_api")

import json
import pybarb as pb

# Set the working directory
working_directory = '/Users/simon_business/Documents/disposable/clients/BARB/'

# Get the access token
with open(working_directory + "creds.json") as file:
    creds = json.load(file)

# Create a BarbAPI object and connect
barb_api = pb.BarbAPI(creds)
barb_api.connect()


## Get data from the API

Let's say as the media agency Mindshare we are interested in the Persil campaign that was shown over for our client Unilever. If we don't have it already, our first use of the API is to find out our own buyer name.

### The buyer name

In [3]:
barb_api.list_buyers("WALKER")

['walker_communication', 'walker_media_limited']

### The advertiser name

We could bring back all the data for Mindshare but it would be better to limit it to the spots that were placed for Unilever. For this we need the right name for Unilever home products. 

In [2]:
barb_api.list_advertisers("ASDA")

['ASDA_STORES']

### The TV station name

Finally, we are only interested in the BBC Network panel so we can use the [panels](https://barb-api.co.uk/api-docs#tag/Metadata/operation/panelsList) end point to get the right code. We need to find the right name for the panel.

In [29]:
barb_api.list_stations("itv")

['ITV1',
 'ITV1 HD',
 'ITV1+1',
 'ITV4+1',
 'ITV Play',
 'ITV2+1',
 'ITV3+1',
 'ITV3',
 'ITV4',
 'CITV',
 'ITV Sport',
 'ITV Sport Select',
 'ITV2',
 'ITV2 HD',
 'ITV Encore',
 'ITV Encore +1',
 'ITVBe',
 'ITVBe +1']

### The panel code

Finally, we are only interested in the BBC Network panel so we can use the [panels](https://barb-api.co.uk/api-docs#tag/Metadata/operation/panelsList) end point to get the right code. We need to find the right name for the panel.

In [ ]:
barb_api.list_panels("bbc")

['BBC Network',
 'BBC East Region',
 'BBC West Region',
 'BBC South West Region',
 'BBC South Region',
 'BBC Yorkshire & Lincolnshire',
 'BBC North East & Cumbria',
 'BBC North West Region',
 'BBC Scotland Region',
 'BBC Ulster Region',
 'BBC Wales Region',
 'BBC Midlands West',
 'BBC Midlands East',
 'BBC London',
 'BBC South East']

### Querying the advertising spots endpoint

Now we know all the relevant metadata we can query the advertising spots endpoint. This can be done very simply using pybarb's `advertising_spots` method. 

In [62]:
spot_data = barb_api.advertising_spots(min_transmission_date = "2022-01-01",
                           max_transmission_date = "2022-04-01", 
                           buyer =  "mindshare_media_uk_l", 
                           advertiser = "EXPEDIA.CO.UK",
                           station = 'ITV1',
                           panel="BBC Network")

In [9]:
spot_data = barb_api.advertising_spots(min_transmission_date = "2022-01-01",
                           max_transmission_date = "2022-04-01", 
                           buyer =  "walker_media_limited", 
                           advertiser = "ASDA_STORES",
                           station = 'ITV1',
                           panel="BBC Network")

## Accessing the data

The raw data is stored in the `api_response_data` attribute of the resulting object (in this case the object named `spot_data`)

In [10]:
spot_data.api_response_data

{'endpoint': 'advertising_spots',
 'events': [{'panel': {'panel_code': 50,
    'panel_region': 'BBC Network',
    'is_macro_region': False},
   'station': {'station_code': 30, 'station_name': 'ITV'},
   'spot_type': 'spot',
   'spot_start_datetime': {'barb_reporting_datetime': '2022-01-25T08:23:58',
    'barb_polling_datetime': '2022-01-25 08:23:58',
    'standard_datetime': '2022-01-25 08:23:58'},
   'spot_duration': 30,
   'preceding_programme_name': 'GOOD MORNING BRITAIN',
   'succeeding_programme_name': None,
   'break_type': 'centre break',
   'broadcaster_spot_number': '68126324',
   'commercial_number': 'HKXASRE039030',
   'clearcast_information': {'match_group_code': 0,
    'match_group_name': 'England, Scotland and Wales',
    'buyer_code': 'B001346',
    'buyer_name': 'WALKER MEDIA LIMITED',
    'advertiser_code': 'A000058',
    'advertiser_name': 'ASDA STORES',
    'holding_company_code': 'H013837',
    'holding_company_name': 'BROADSTREET GREAT WI',
    'product_code': 'P10

However it is easier to access it as a dataframe. To do this, we can use the `to_dataframe()` method, which flattens the nested json structure.

In [65]:
spot_df = spot_data.to_dataframe()
spot_df


,panel_region,station_name,spot_type,spot_start_datetime,spot_duration,preceding_programme_name,succeeding_programme_name,break_type,position_in_break,broadcaster_spot_number,...,clearcast_buyer_code,clearcast_buyer_name,clearcast_advertiser_code,clearcast_advertiser_name,campaign_approval_id,sales_house_name,audience_size_hundreds,date_of_transmission,audience_name,audience_target_size_hundreds
0,BBC Network,ITV,spot,2022-01-25 07:36:42,10,GOOD MORNING BRITAIN,None,centre break,other,68110749,...,B006355,MEDIA AGENCY GROUP L,A015982,SYKES COTTAGES,1034138.0,ITV Breakfast,2717,2022-01-25,All Homes,270570
1,BBC Network,ITV,spot,2022-01-25 07:36:42,10,GOOD MORNING BRITAIN,None,centre break,other,68110749,...,B006355,MEDIA AGENCY GROUP L,A015982,SYKES COTTAGES,1034138.0,ITV Breakfast,2879,2022-01-25,All Adults,512010
2,BBC Network,ITV,spot,2022-01-25 07:36:42,10,GOOD MORNING BRITAIN,None,centre break,other,68110749,...,B006355,MEDIA AGENCY GROUP L,A015982,SYKES COTTAGES,1034138.0,ITV Breakfast,1278,2022-01-25,All Men,249750
3,BBC Network,ITV,spot,2022-01-25 07:36:42,10,GOOD MORNING BRITAIN,None,centre break,other,68110749,...,B006355,MEDIA AGENCY GROUP L,A015982,SYKES COTTAGES,1034138.0,ITV Breakfast,1858,2022-01-25,All Houseperson,270570
4,BBC Network,ITV,spot,2022-01-25 07:36:42,10,GOOD MORNING BRITAIN,None,centre break,other,68110749,...,B006355,MEDIA AGENCY GROUP L,A015982,SYKES COTTAGES,1034138.0,ITV Breakfast,79,2022-01-25,All Children aged 4-15,95390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146919,BBC Network,ITV,spot,2022-03-27 09:23:35,30,DONT UNLEASH THE BEAST,CLOSEDOWN,end break,first,68702590,...,B001033,STARCOM WORLDWIDE LI,A000375,PROCTER & GAMBLE UK,1051928.0,ITV Breakfast,88,2022-03-27,Houseperson with children 0-15,69670
146920,BBC Network,ITV,spot,2022-03-27 09:23:35,30,DONT UNLEASH THE BEAST,CLOSEDOWN,end break,first,68702590,...,B001033,STARCOM WORLDWIDE LI,A000375,PROCTER & GAMBLE UK,1051928.0,ITV Breakfast,138,2022-03-27,Children 04-09,48160
146921,BBC Network,ITV,spot,2022-03-27 09:23:35,30,DONT UNLEASH THE BEAST,CLOSEDOWN,end break,first,68702590,...,B001033,STARCOM WORLDWIDE LI,A000375,PROCTER & GAMBLE UK,1051928.0,ITV Breakfast,95,2022-03-27,Children 04-06,23880
146922,BBC Network,ITV,spot,2022-03-27 09:23:35,30,DONT UNLEASH THE BEAST,CLOSEDOWN,end break,first,68702590,...,B001033,STARCOM WORLDWIDE LI,A000375,PROCTER & GAMBLE UK,1051928.0,ITV Breakfast,96,2022-03-27,Boys 04-15,49440


In [61]:
spot_data.to_csv('spot_data.csv')

## Manipulating the data

We can also get a pivot of the data which turns the audiences into columns.

In [57]:
spot_data.audience_pivot()

audience_name                                                                 ABC1 Adults, Lightest Third   
panel_region station_name date_of_transmission clearcast_commercial_title                                   
BBC Network  ITV          2022-01-01           ACTIVATE Generic                                       0.0  \
                                               ACTIVATE TUI                                           0.0   
                                               BBA NOA (ARGOS)                                        0.0   
                                               Buster Bathroom Unblocker 10                           0.0   
                                               GHR Simon Mayo V1 London                               0.0   
...                                                                                                   ...   
                          2022-04-01           Vax Blade Range 4 Updates Feb                          0.0   
                                               WPPEMMA113020                                         79.0   
                                               WPPEMMA114010                                         25.0   
                                               Walkers Holidays 20                                   79.0   
                                               Wrong Van                                             79.0   

audience_name                                                                 Adults 16-24   
panel_region station_name date_of_transmission clearcast_commercial_title                    
BBC Network  ITV          2022-01-01           ACTIVATE Generic                        0.0  \
                                               ACTIVATE TUI                            0.0   
                                               BBA NOA (ARGOS)                         0.0   
                                               Buster Bathroom Unblocker 10            0.0   
                                               GHR Simon Mayo V1 London                0.0   
...                                                                                    ...   
                          2022-04-01           Vax Blade Range 4 Updates Feb           0.0   
                                               WPPEMMA113020                           0.0   
                                               WPPEMMA114010                           0.0   
                                               Walkers Holidays 20                     0.0   
                                               Wrong Van                               0.0   

audience_name                                                                 Adults 16-34   
panel_region station_name date_of_transmission clearcast_commercial_title                    
BBC Network  ITV          2022-01-01           ACTIVATE Generic                        0.0  \
                                               ACTIVATE TUI                            0.0   
                                               BBA NOA (ARGOS)                        48.0   
                                               Buster Bathroom Unblocker 10            0.0   
                                               GHR Simon Mayo V1 London                0.0   
...                                                                                    ...   
                          2022-04-01           Vax Blade Range 4 Updates Feb         430.0   
                                               WPPEMMA113020                          53.0   
                                               WPPEMMA114010                         315.0   
                                               Walkers Holidays 20                    53.0   
                                               Wrong Van                              53.0   

audience_name                                                                 Adults 16-34, Lightest Third   
panel_region station_name 

In [51]:
spot_df.clearcast_advertiser_name.value_counts().head(30).index

Index(['LOREAL UK', 'RECKITT BENCKISER', 'BOXT', 'UNIVERSAL THEATRICAL',
       'GOHENRY', 'PROCTER & GAMBLE UK', 'GOUSTO', 'EURO PRO EUROPE',
       'BOURNE LEISURE', 'TESCO STORES', 'CHARACTER OPTIONS', 'COTY UK',
       'VIRGIN MEDIA', 'AGE PARTNERSHIP', 'EMMA MATTRESS', 'VAX', 'BISL',
       'DOGMATES', 'KSF ACQUISITION UK', 'OMAZE', 'EXPEDIA.CO.UK',
       'CAMELOT GROUP', 'LINTBELLS', 'JANE PLAN', 'TURKISH AIRLINES',
       'GO DADDY', 'BRITISH TOY & HOBBY', 'DIGITAL MONEYBOX',
       'SMILEDIRECTCLUB', 'CCS'],
      dtype='object', name='clearcast_advertiser_name')

In [58]:
import plotly.express as px
big_advertisers = ['LOREAL UK', 'RECKITT BENCKISER', 'BOXT', 'UNIVERSAL THEATRICAL',
       'GOHENRY', 'PROCTER & GAMBLE UK']
spot_df = spot_data.to_dataframe()
persil_sky = spot_df[(spot_df['clearcast_advertiser_name'].isin(big_advertisers)) 
                   & (spot_df['audience_name']=='All Homes')].sort_values(['spot_start_datetime'])

fig = px.scatter(persil_sky, x="spot_start_datetime", y="audience_size_hundreds", color="clearcast_advertiser_name", width = 1400, height = 600)
fig.update_traces(marker={'size': 10})

shapes=[dict(
        type='line',
        x0 = persil_sky['spot_start_datetime'].iloc[i],
        y0 = 0,
        x1 = persil_sky['spot_start_datetime'].iloc[i],
        y1 = persil_sky['audience_size_hundreds'].iloc[i],
        layer='below',
        line = dict(
            color = 'grey',
            width = 2
        )
    ) for i in range(len(persil_sky['audience_size_hundreds']))]

fig.update_layout(shapes = shapes)

In [41]:
spot_df.columns

Index(['panel_region', 'station_name', 'spot_type', 'spot_start_datetime',
       'spot_duration', 'preceding_programme_name',
       'succeeding_programme_name', 'break_type', 'position_in_break',
       'broadcaster_spot_number', 'commercial_number',
       'clearcast_commercial_title', 'clearcast_match_group_code',
       'clearcast_match_group_name', 'clearcast_buyer_code',
       'clearcast_buyer_name', 'clearcast_advertiser_code',
       'clearcast_advertiser_name', 'campaign_approval_id', 'sales_house_name',
       'audience_size_hundreds', 'date_of_transmission', 'audience_name',
       'audience_target_size_hundreds'],
      dtype='object')